In [ ]:
import pandas as pd
import numpy as np
import skimage.io as sio
import numpy.ma as ma
from scipy import interpolate

import diff_classifier.aws as aws
import diff_classifier.utils as ut
import diff_classifier.msd as msd
import diff_classifier.features as ft

In [ ]:
files = !ls Traj*
local_name = files[0]
test = ut.csv_to_pd(local_name)

In [ ]:
test

In [ ]:
def make_xyarray(data):
    
    #Initial values
    length = 651
    particles = max(data['Track_ID'])
    x_array = np.zeros((length, particles+1))
    y_array = np.zeros((length, particles+1))
    f_array = np.zeros((length, particles+1))
    t_array = np.zeros((length, particles+1))

    track = data[data['Track_ID']==0].sort_values(['Track_ID', 'Frame'], ascending=[1, 1]).reset_index(drop=True)
    new_frame = np.linspace(0, length-1, length)

    old_frame = track['Frame']
    old_x = track['X']
    old_y = track['Y']
    fx = interpolate.interp1d(old_frame, old_x, bounds_error = False, fill_value = np.nan)
    fy = interpolate.interp1d(old_frame, old_y, bounds_error = False, fill_value = np.nan)

    int_x = fx(new_frame)
    int_y = fy(new_frame)
    
    #Fill in entire array
    x_array[:, 0] = int_x
    y_array[:, 0] = int_y
    f_array[:, 0] = new_frame
    t_array[:, 0] = 0

    for part in range(first_p+1, particles+1):
        track = data[data['Track_ID']==part].sort_values(['Track_ID', 'Frame'], ascending=[1, 1]).reset_index(drop=True)

        old_frame = track['Frame']
        old_x = track['X']
        old_y = track['Y']
        fx = interpolate.interp1d(old_frame, old_x, bounds_error = False, fill_value = np.nan)
        fy = interpolate.interp1d(old_frame, old_y, bounds_error = False, fill_value = np.nan)

        int_x = fx(new_frame)
        int_y = fy(new_frame)

        x_array[:, part] = int_x
        y_array[:, part] = int_y
        f_array[:, part] = new_frame
        t_array[:, part] = part
    
    return f_array, t_array, x_array, y_array

In [ ]:
def all_msds2(data):

    f_array, t_array, x_array, y_array = make_xyarray(data)

    length = x_array.shape[0]
    particles = x_array.shape[1]

    MSD = np.zeros((length, particles))
    gauss = np.zeros((length, particles))

    for frame in range(0, length-1):
        x = np.square(msd.nth_diff(x_array, n=frame+1))
        y = np.square(msd.nth_diff(y_array, n=frame+1))

        MSD[frame+1, :] = np.nanmean(x + y, axis=0)
        gauss[frame+1, :] = np.nanmean(x**2 + y**2, axis=0)/(2*(MSD[frame+1]**2))
    
    d = {'Frame': f_array.flatten('F'),
         'Track_ID': t_array.flatten('F'),
         'X': x_array.flatten('F'),
         'Y': y_array.flatten('F'),
         'MSDs': MSD.flatten('F'),
         'Gauss': gauss.flatten('F')}
    new_data = pd.DataFrame(data=d)
    
    return new_data

In [ ]:
new_data = all_msds2(test)

In [ ]:
new_data[new_data['Track_ID']==100]